In [4]:
import requests
import pandas as pd
#import juypter notebooks
import import_ipynb

from bs4 import BeautifulSoup
from collections import defaultdict #This means that a Python Tuple can be a key whereas a Python List can not
#https://www.geeksforgeeks.org/defaultdict-in-python/

from unidecode import unidecode #remove non-ascii characters from a text document
from sklearn import svm
#ignore warrnings 
import warnings
warnings.filterwarnings('ignore')

In [2]:
#get team table from wikipedia.
team_table_page = requests.get('https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Basketball_Association/National_Basketball_Association_team_abbreviations')
soup = BeautifulSoup(team_table_page.text)
team_table = {}
for row in soup.find_all('tr'):
    entries = row.find_all('td')
    team_table[entries[1].text.strip()] = entries[0].text.strip()
    
team_table

{'Franchise': 'Abbreviation/Acronym',
 'Atlanta Hawks': 'ATL',
 'Brooklyn Nets': 'BKN',
 'Boston Celtics': 'BOS',
 'Charlotte Hornets': 'CHA',
 'Chicago Bulls': 'CHI',
 'Cleveland Cavaliers': 'CLE',
 'Dallas Mavericks': 'DAL',
 'Denver Nuggets': 'DEN',
 'Detroit Pistons': 'DET',
 'Golden State Warriors': 'GSW',
 'Houston Rockets': 'HOU',
 'Indiana Pacers': 'IND',
 'Los Angeles Clippers': 'LAC',
 'Los Angeles Lakers': 'LAL',
 'Memphis Grizzlies': 'MEM',
 'Miami Heat': 'MIA',
 'Milwaukee Bucks': 'MIL',
 'Minnesota Timberwolves': 'MIN',
 'New Orleans Pelicans': 'NOP',
 'New York Knicks': 'NYK',
 'Oklahoma City Thunder': 'OKC',
 'Orlando Magic': 'ORL',
 'Philadelphia 76ers': 'PHI',
 'Phoenix Suns': 'PHX',
 'Portland Trail Blazers': 'POR',
 'Sacramento Kings': 'SAC',
 'San Antonio Spurs': 'SAS',
 'Toronto Raptors': 'TOR',
 'Utah Jazz': 'UTA',
 'Washington Wizards': 'WAS'}

In [3]:
#new_key = "New Jersey Nets"
#old_key = "Brooklyn Nets"
#team_table[new_key] = team_table.pop(old_key)

print(team_table)


{'Franchise': 'Abbreviation/Acronym', 'Atlanta Hawks': 'ATL', 'Brooklyn Nets': 'BKN', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHA', 'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Dallas Mavericks': 'DAL', 'Denver Nuggets': 'DEN', 'Detroit Pistons': 'DET', 'Golden State Warriors': 'GSW', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND', 'Los Angeles Clippers': 'LAC', 'Los Angeles Lakers': 'LAL', 'Memphis Grizzlies': 'MEM', 'Miami Heat': 'MIA', 'Milwaukee Bucks': 'MIL', 'Minnesota Timberwolves': 'MIN', 'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC', 'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHX', 'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS', 'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS'}


### Extract win prc rate for each team from basketball-reference.com/
At this part i convert the win html table into BeautifulSoup format.

i took all the win percentage for each team all over the years since 2015-2021

Each table is located in different link so i had to change the url link for each year.

In [4]:
listofyears = ["2015", "2016", "2017", "2018", "2019", 
               "2020", "2021"]

In [5]:
# get all season win percentages since 2015
wins = {}
for year in range(2015, 2022):
    print(year)
    team_wins = {}
    win_html = requests.get('https://www.basketball-reference.com/leagues/NBA_{}.html'.format(year)).text
    bs = BeautifulSoup(win_html)
    tbls = bs.find_all('table')
    east = tbls[0]
    west = tbls[1]
    for row in east.find_all('tr'):
        head = row.find_all('th')
        data = row.find_all('td')
        if (len(data) > 0):
            team_name = team_table[head[0].text.replace('*', '')]
            win_perc = float(data[2].text)
            team_wins[team_name] = win_perc

    for row in west.find_all('tr'):
        head = row.find_all('th')
        data = row.find_all('td')
        if (len(data) > 0):
            team_name = team_table[head[0].text.replace('*', '')]
            win_perc = float(data[2].text)
            team_wins[team_name] = win_perc
    
    wins[year] = team_wins

2015
2016
2017
2018
2019
2020
2021


### Scarpe stats and clear some of the details
This function gets a page and return all details i need like : stats per game, advanced stats, clean them and check if there is any player who play in more than one team ("TOT"), and using other function that fix player name

In [6]:
def get_nba_df(year):
    df = pd.read_html("https://www.basketball-reference.com/leagues/NBA_"+str(year)+"_per_game.html")[0]

    adv_df = pd.read_html("https://www.basketball-reference.com/leagues/NBA_"+str(year)+"_advanced.html")[0]
    df = df.combine_first(adv_df)
    df = df.astype('string')
    is_tot = df['Tm']=='TOT'
    traded_players = df[is_tot]['Player']
    # dont record half-seasons of traded players, only total seasons ("TOT")
    

    df = df.query('not (Player in @traded_players & Tm != "TOT")')
    df.fillna("0", inplace=True)
    df.replace(r'^\s*$', "0", regex=True, inplace=True)
    # filter out header values
    df = df[(df['2P'] != "2P")]
    df = df[(df['PF'] != "PF")]
    df = df.astype({'Pos': 'string', '2P': 'float', '2PA': 'float', '3P': 'float', '3P%': 'float', '3PA': 'float', '3PAr': 'float', 'AST': 'float',
                    'AST%': 'float', 'Age': 'int', 'BLK': 'float', 'BLK%': 'float', 'BPM': 'float', 'DBPM': 'float', 
                    'DRB': 'float', 'DRB%': 'float', 'DWS': 'float', 'FG': 'float', 'FG%': 'float', 'FGA': 'float',
                    'FT': 'float', 'FT%': 'float', 'FTA': 'float', 'FTr': 'float', 'G': 'int', 'GS': 'int', 'MP': 'float', 
                    'OBPM': 'float', 'ORB': 'float', 'ORB%': 'float', 'OWS': 'float', 'PER': 'float', 'PF': 'float', 'PTS': 'float','Rk': 'float', 'STL': 'float', 'STL%': 'float', 'TOV': 'float', 'TOV%': 'float', 
                    'TRB': 'float', 'TRB%': 'float', 'TS%': 'float', 'USG%': 'float', 'WS': 'float', 'eFG%': 'float', 'VORP': 'float'})
    df['Player'] = df['Player'].apply(fix_name)
    return df

### Scrape Players who play in ALL STAR GAMES and fix worng data

In [7]:
# i found on post https://towardsdatascience.com/using-machine-learning-to-predict-nba-all-stars-part-1-data-collection-9fb94d386530
# a way of making the prediction more accurate, i changed the way that data was collected mapping year -> all star instead of the other way around
# this dictionary will map year to players in all star game
# and i selected for an all-star game, either initially or as a replacement

all_star_appearances = defaultdict(list) # using this article https://www.geeksforgeeks.org/defaultdict-in-python/

# rows to ignore when iterating the roster tables
ignore_fields = set(['Team Totals', 'Reserves'])

START_YEAR, END_YEAR = 2015, 2022 #Not including 2022 cause this my prediction)

# unidecode doesn't catch the accented c in Peja's last name (Stojakovic), fix it
# also overwrite any instance of Metta World Peace to Ron Artest
def fix_name(full_name):
    first_name = full_name.split(' ')[0]
    if first_name == 'Peja':
        return 'Peja Stojakovic'
    elif first_name == 'Metta':
        return 'Ron Artest'
    else:
        return unidecode(full_name)

for year in range(START_YEAR, END_YEAR):

    print('Scraping All Star Game {} data...'.format(year))

    # will store all the all-stars for this year
    all_stars = set([])

    html = requests.get('https://www.basketball-reference.com/allstar/NBA_{}.html'.format(year)).content
    soup = BeautifulSoup(html, 'html.parser')

    # were encoded with id="East"/id="West" so they could be extracted more easily/reliably
    
    # gets rosters for team 1 and team 2
    s1, s2 = soup.findAll('table')[1:3]

    df1 = pd.read_html(str(s1))[0]
    df2 = pd.read_html(str(s2))[0]

    # get the all-stars from teams 1 and 2
    for df in [df1, df2]:
        for i, row in df.iterrows():
            if pd.notnull(row[0]) and row[0] not in ignore_fields:
                player = row[0]
                all_stars.add(fix_name(player))

    # gets all li elements in the page
    s3 = soup.findAll('li') 

    # finds the li element that contains the data pertaining to injury related selections 
    # - players who were selected but couldn't participate due to injury,
    # and their respective replacements
    #
    # since all_stars is a hashset, we don't need to worry about accidentally double counting an all-star
    for s in s3:
        if 'Did not play' in str(s):
            for player in [name.get_text() for name in s.findAll('a')]: # all the injured players and their replacements
                all_stars.add(fix_name(player))
            break

    # update the appearances dictionary
    for player in all_stars:
        all_star_appearances[year].append(player)

Scraping All Star Game 2015 data...
Scraping All Star Game 2016 data...
Scraping All Star Game 2017 data...
Scraping All Star Game 2018 data...
Scraping All Star Game 2019 data...
Scraping All Star Game 2020 data...
Scraping All Star Game 2021 data...


In [8]:
all_star_appearances[2021]

['Bradley Beal',
 'Kawhi Leonard',
 'Jayson Tatum',
 'Zion Williamson',
 'Kevin Durant',
 'Ben Simmons',
 'Giannis Antetokounmpo',
 'Stephen Curry',
 'Domantas Sabonis',
 'Rudy Gobert',
 'Kyrie Irving',
 'Paul George',
 'Julius Randle',
 'Anthony Davis',
 'Joel Embiid',
 'Jaylen Brown',
 'Luka Doncic',
 'LeBron James',
 'Devin Booker',
 'Damian Lillard',
 'Zach LaVine',
 'Chris Paul',
 'Mike Conley',
 'Nikola Vucevic',
 'Donovan Mitchell',
 'James Harden',
 'Nikola Jokic']

this functuon read from html and put in in dataframe

In [9]:
def get_team_pct(year, row):
    if team == 'TOT':
        return wins[year][row['Tm']]

calling another function that get details from all star games, and now 
in df we handle the dataframe, and making a array of dataframes

In [11]:
year_dfs = []
for year in range(2015, 2022):
    df = get_nba_df(year) #calling this function to get details
    # Add all star field
    as_data = []

    for player in all_star_appearances[year]:
        as_data.append([player, True])
        
        
    as_df = pd.DataFrame(as_data, columns=['Player', 'All_Star'])
    df = pd.merge(df, as_df, left_on='Player', right_on='Player', how='left')
    df = df.astype({'All_Star': 'boolean'})
    df['All_Star'] = df['All_Star'].fillna(False)
    
    
    year_dfs.append(df)

Now i "merge" all the dataframe into one dataframe.

In [12]:
df

,2P,2P%,2PA,3P,3P%,3PA,3PAr,AST,AST%,Age,...,TS%,Tm,USG%,Unnamed: 19,Unnamed: 24,VORP,WS,WS/48,eFG%,All_Star
0,2.0,.546,3.7,0.0,0.000,0.0,0.004,0.5,6.1,21,...,0.550,MIA,19.5,0,0,-0.4,1.3,.085,0.544,False
1,0.1,.167,0.9,0.0,0.000,0.3,0.250,0.3,12.7,24,...,0.125,MIL,18.6,0,0,-0.1,-0.1,-0.252,0.125,False
2,3.3,.620,5.3,0.0,0.000,0.1,0.010,1.9,9.1,27,...,0.596,NOP,11.7,0,0,0.7,4.0,.119,0.614,False
3,7.1,.573,12.4,0.0,0.250,0.1,0.010,5.4,26.9,23,...,0.626,MIA,23.7,0,0,3.7,8.8,.197,0.571,False
4,4.2,.505,8.3,1.2,0.388,3.1,0.270,1.9,11.0,35,...,0.556,TOT,22.2,0,0,0.3,1.1,.080,0.525,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,2.8,.509,5.5,1.0,0.372,2.7,0.332,4.4,23.0,28,...,0.562,TOT,16.3,0,0,1.8,4.3,.117,0.525,False
536,5.3,.580,9.1,0.2,0.267,0.7,0.068,4.3,27.1,32,...,0.578,CHI,22.3,0,0,2.2,5.1,.147,0.568,False
537,5.6,.491,11.3,2.2,0.343,6.3,0.357,9.4,45.5,22,...,0.589,ATL,33.0,0,0,3.0,7.2,.163,0.499,False
538,3.7,.598,6.2,0.1,0.143,0.6,0.086,1.8,13.2,28,...,0.599,CHO,18.3,0,0,0.4,3.3,.156,0.565,False


In [13]:
#Get final data by concatenating all years
final = pd.concat(year_dfs)
#final.reset_index(drop=True)
# Save final data
final = final.drop(['Unnamed: 19', 'Unnamed: 24'], axis=1)
final.to_csv('data/All_Star_Data.csv')

In [14]:
#TEST TO CHECK IF EVERYTHING OK
final = pd.read_csv('data/All_Star_Data.csv')

print(final[final['All_Star']==True]['PTS'].quantile(0.01))
print(final[final['All_Star']==True]['AST'].quantile(0.01))
print(final[final['All_Star']==True]['USG%'].quantile(0.01))

10.84
1.12
16.02


In [15]:
final

,Unnamed: 0,2P,2P%,2PA,3P,3P%,3PA,3PAr,AST,AST%,...,TRB,TRB%,TS%,Tm,USG%,VORP,WS,WS/48,eFG%,All_Star
0,0,2.0,0.494,4.0,0.3,0.300,0.9,0.181,1.0,8.7,...,4.4,13.5,0.533,NYK,15.5,-0.3,1.7,0.063,0.486,False
1,1,0.8,0.410,2.0,0.3,0.400,0.8,0.291,0.5,10.1,...,0.9,6.4,0.489,MEM,20.4,0.2,0.4,0.073,0.465,False
2,2,3.1,0.547,5.7,0.0,0.000,0.0,0.005,0.9,5.5,...,7.5,15.8,0.549,OKC,14.3,0.3,4.1,0.111,0.544,False
3,3,1.1,0.432,2.6,0.0,0.000,0.0,0.000,0.9,10.5,...,4.5,20.5,0.494,MIN,14.3,0.0,0.4,0.087,0.432,False
4,4,3.3,0.466,7.1,1.5,0.354,4.3,0.377,1.7,8.2,...,3.2,5.3,0.533,TOT,19.0,-0.3,2.6,0.050,0.491,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3588,535,2.8,0.509,5.5,1.0,0.372,2.7,0.332,4.4,23.0,...,4.3,8.6,0.562,TOT,16.3,1.8,4.3,0.117,0.525,False
3589,536,5.3,0.580,9.1,0.2,0.267,0.7,0.068,4.3,27.1,...,6.2,14.2,0.578,CHI,22.3,2.2,5.1,0.147,0.568,False
3590,537,5.6,0.491,11.3,2.2,0.343,6.3,0.357,9.4,45.5,...,3.9,6.3,0.589,ATL,33.0,3.0,7.2,0.163,0.499,False
3591,538,3.7,0.598,6.2,0.1,0.143,0.6,0.086,1.8,13.2,...,6.8,17.7,0.599,CHO,18.3,0.4,3.3,0.156,0.565,False


In [16]:
final.to_csv('data/All_Star_Data.csv')

## Scrape Data for 2022 for predictions

In [33]:
stats_pct = ['3P%', 'AST%', 'Age', 'BLK%', 'eFG%', 'FT%', 
                    'G', 'PTS', 'STL%', 'DBPM', 'OBPM',
                    'ORB%', 'DRB%', 'USG%', 'TOV%']
# and the per game numbers
stats_pg = ['3P%', 'AST', 'Age', 'BLK', 'eFG%', 'FT%', 
                    'G',  'PTS', 'STL', 'DBPM', 
                  'OBPM', 'ORB', 'DRB', 'PF',  'USG%', 'TOV', 'FTr']
df = get_nba_df(2022)
df = df[df['G'] > 10]

In [34]:
df

,2P,2P%,2PA,3P,3P%,3PA,3PAr,AST,AST%,Age,...,TRB%,TS%,Tm,USG%,Unnamed: 19,Unnamed: 24,VORP,WS,WS/48,eFG%
0,2.9,.415,7.0,0.3,0.290,1.2,0.145,1.5,8.9,22,...,16.6,0.439,TOR,17.7,0,0,-0.3,0.5,.039,0.418
1,2.6,.530,5.0,0.0,0.000,0.0,0.000,3.0,15.1,28,...,19.5,0.561,MEM,12.7,0,0,0.8,3.2,.150,0.530
2,7.0,.521,13.4,0.0,0.000,0.1,0.004,3.2,16.0,24,...,17.3,0.577,MIA,25.8,0,0,0.6,1.8,.147,0.519
3,1.3,.543,2.3,0.2,0.121,1.7,0.418,0.7,8.1,21,...,14.2,0.416,MEM,18.0,0,0,-0.3,0.1,.032,0.392
4,5.4,.596,9.1,0.4,0.343,1.3,0.121,0.8,5.9,36,...,13.1,0.614,BRK,22.4,0,0,0.6,2.3,.167,0.586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,2.8,.586,4.8,0.0,0.000,0.1,0.026,2.4,24.4,33,...,12.7,0.566,SAS,19.0,0,0,0.4,1.0,.147,0.570
633,6.9,.502,13.8,2.8,0.381,7.4,0.350,9.5,48.1,23,...,6.1,0.588,ATL,35.4,0,0,2.0,4.3,.180,0.527
634,2.3,.496,4.7,0.0,0.143,0.2,0.047,0.7,8.0,23,...,23.8,0.505,MIA,19.8,0,0,-0.1,1.0,.118,0.483
635,1.8,.597,3.0,0.0,0.000,0.2,0.053,0.8,9.3,29,...,19.1,0.631,POR,15.5,0,0,0.0,1.0,.154,0.566


In [35]:
df = df.drop(['Unnamed: 19', 'Unnamed: 24'], axis=1)

In [36]:
df

,2P,2P%,2PA,3P,3P%,3PA,3PAr,AST,AST%,Age,...,TOV%,TRB,TRB%,TS%,Tm,USG%,VORP,WS,WS/48,eFG%
0,2.9,.415,7.0,0.3,0.290,1.2,0.145,1.5,8.9,22,...,11.3,7.6,16.6,0.439,TOR,17.7,-0.3,0.5,.039,0.418
1,2.6,.530,5.0,0.0,0.000,0.0,0.000,3.0,15.1,28,...,20.5,9.4,19.5,0.561,MEM,12.7,0.8,3.2,.150,0.530
2,7.0,.521,13.4,0.0,0.000,0.1,0.004,3.2,16.0,24,...,15.3,10.2,17.3,0.577,MIA,25.8,0.6,1.8,.147,0.519
3,1.3,.543,2.3,0.2,0.121,1.7,0.418,0.7,8.1,21,...,7.5,2.9,14.2,0.416,MEM,18.0,-0.3,0.1,.032,0.392
4,5.4,.596,9.1,0.4,0.343,1.3,0.121,0.8,5.9,36,...,7.4,5.8,13.1,0.614,BRK,22.4,0.6,2.3,.167,0.586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,2.8,.586,4.8,0.0,0.000,0.1,0.026,2.4,24.4,33,...,15.9,3.4,12.7,0.566,SAS,19.0,0.4,1.0,.147,0.570
633,6.9,.502,13.8,2.8,0.381,7.4,0.350,9.5,48.1,23,...,14.2,3.9,6.1,0.588,ATL,35.4,2.0,4.3,.180,0.527
634,2.3,.496,4.7,0.0,0.143,0.2,0.047,0.7,8.0,23,...,13.7,6.0,23.8,0.505,MIA,19.8,-0.1,1.0,.118,0.483
635,1.8,.597,3.0,0.0,0.000,0.2,0.053,0.8,9.3,29,...,15.0,4.6,19.1,0.631,POR,15.5,0.0,1.0,.154,0.566


In [37]:
df.to_csv('data/Stats2022.csv')

In [5]:
AllStar_df = pd.read_csv("data/All_Star_Data.csv")

In [6]:
AllStar_df["All_Star"].value_counts()

False    3412
True      181
Name: All_Star, dtype: int64